In [15]:
# Tratamiento de datos
# -----------------------------------------------------------------------
import numpy as np
import pandas as pd

# Gráficos
# ------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns


#  Modelado y evaluación
# -----------------------------------------------------------------------------
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

import warnings
warnings.filterwarnings('ignore')

plt.rcParams["figure.figsize"] = (20,10) 
pd.options.display.max_columns = None

In [16]:
df= pd.read_pickle('datos/cars_price_norm_est_cod.pkl')
df.head(2)

,Levy,Prod_year,Mileage,Cylinders,Airbags,Price_log,Leather_interior_le,Fuel_type_CNG,Fuel_type_Diesel,Fuel_type_Hybrid,Fuel_type_Hydrogen,Fuel_type_LPG,Fuel_type_Petrol,Fuel_type_Plug-in Hybrid,Gear_box_type_Automatic,Gear_box_type_Manual,Gear_box_type_Tiptronic,Gear_box_type_Variator,Drive_wheels_4x4,Drive_wheels_Front,Drive_wheels_Rear,Category_Cabriolet,Category_Coupe,Category_Goods wagon,Category_Hatchback,Category_Jeep,Category_Limousine,Category_Microbus,Category_Minivan,Category_Pickup,Category_Sedan,Category_Universal
0,1.766963,-0.411248,0.237134,1.375313,1.329406,9.497622,1,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0
1,0.943936,-0.173337,0.273438,1.375313,0.319668,9.718422,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0


In [17]:
df.shape

(9877, 32)

##### <font color=green> En el pair programming anterior creastéis vuestro primer modelo de machine learning usando la regresion Lineal. Es el momento, que con vuestros datos evaluéis si es bueno haciendo predicciones. Los objetivo de este pairprogramming son:

##### <font color=green> Calculéis las métricas para vuestro modelo

- Primero realizaremos la creación del modelo, igual que realizamos en el pair anterior

In [18]:
X = df.drop("Price_log", axis = 1)
y = df["Price_log"]

In [19]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [20]:
# iniciamos la regresión lineal. n_jobs hace referencia al número de nucleos que usaremos de nuestro ordenador. Al indicar -1 significa que los usaremos todos. 

lr = LinearRegression(n_jobs=-1)

In [21]:
# fiteamos el modelo, lo que significa que le pasamos los datos de entrenamiento para que aprenda el algoritmo. Fijaros que para que aprenda, les paso solo los datos de entrenamiento

lr.fit(x_train, y_train)

LinearRegression(n_jobs=-1)

In [22]:
# es el momento de hacer las predicciones, para eso usarmos el método predict() de sklearn 

y_predict_train = lr.predict(x_train) # hacemos las predicciones para los coches que tenemos en el los datos de entrenamiento
y_predict_test = lr.predict(x_test) # hacemos las predicciones para los coches que tenemos en los datos de test

In [23]:
train_df = pd.DataFrame({'Real': y_train, 'Predicted': y_predict_train, 'Set': ['Train']*len(y_train)})
test_df  = pd.DataFrame({'Real': y_test,  'Predicted': y_predict_test,  'Set': ['Test']*len(y_test)})
resultados = pd.concat([train_df,test_df], axis = 0)
resultados.head()

,Real,Predicted,Set
13506,7.619724,9.199463,Train
1284,9.404508,9.326050,Train
278,10.816934,10.130859,Train
12539,9.303466,9.416870,Train
7494,10.810738,10.117676,Train


In [24]:
resultados['residuos'] = resultados['Real'] - resultados['Predicted']
resultados.head()

,Real,Predicted,Set,residuos
13506,7.619724,9.199463,Train,-1.579739
1284,9.404508,9.326050,Train,0.078458
278,10.816934,10.130859,Train,0.686075
12539,9.303466,9.416870,Train,-0.113404
7494,10.810738,10.117676,Train,0.693062


### <font color = violet>Cálculo de las métricas

In [25]:
# Para el conjunto de entrenamsiento las méticas han sido
print('Para el conjunto train:---------------')
print('El valor de r2 score es ',r2_score(y_train,y_predict_train))
print('El MAE es',mean_absolute_error(y_train,y_predict_train))
print('El MSE es',mean_squared_error(y_train,y_predict_train))
print('EL RMSE es ',np.sqrt(mean_squared_error(y_train,y_predict_train)))


# Para el conjunto de test las métricas han sido
print('Para el conjunto test:---------------')
print('El valor de r2 score es ',r2_score(y_test,y_predict_test))
print('El MAE es',mean_absolute_error(y_test,y_predict_test))
print('El MSE es',mean_squared_error(y_test,y_predict_test))
print('EL RMSE es ',np.sqrt(mean_squared_error(y_test,y_predict_test)))

Para el conjunto train:---------------
El valor de r2 score es  0.15128266174858473
El MAE es 0.6598271621690035
El MSE es 0.8415465397793868
EL RMSE es  0.9173584576267811
Para el conjunto test:---------------
El valor de r2 score es  -1.3854977560164072e+20
El MAE es 262138495.7465221
El MSE es 1.3578398305239764e+20
EL RMSE es  11652638458.838308


Lo convertimos a dataframe para analizarlo mejor:

In [26]:
resultados_metricas = {'MAE': [mean_absolute_error(y_test, y_predict_test), mean_absolute_error(y_train, y_predict_train)],
                'MSE': [mean_squared_error(y_test, y_predict_test), mean_squared_error(y_train, y_predict_train)],
                'RMSE': [np.sqrt(mean_squared_error(y_test, y_predict_test)), np.sqrt(mean_squared_error(y_train, y_predict_train))],
                'R2':  [r2_score(y_test, y_predict_test), r2_score(y_train, y_predict_train)],
                 "set": ["test", "train"], 
                 "modelo": ["Linear Regresion", "LinearRegression"]}

df_resultados = pd.DataFrame(resultados_metricas)

df_resultados

,MAE,MSE,RMSE,R2,set,modelo
0,2.621385e+08,1.357840e+20,1.165264e+10,-1.385498e+20,test,Linear Regresion
1,6.598272e-01,8.415465e-01,9.173585e-01,1.512827e-01,train,LinearRegression


##### <font color=green> Discutid los resultados de las métricas y extraed conclusiones

Análisis de resultados.

- R²: en el train tenemos un valor de 0.15128266174858473, nuestro modelo no es capaz de explicar ni el 1% de la variación en el valor de los precios de los coches. Es decir, nuestras variables predictoras explican casi nada la variación de los precios de los coches. Realmente es un valor muy bajo y muy malo, habría que incluir otras variables predictoras a nuestro modelo, pero cuáles?

- MAE, el MSE y RMSE: Estos valores están muy lejos de cero y más cerca de 1, lo que nos indica que nuestro modelo comete muchos errores en las predicciones, es decir, nuestro modelo no es capaz de ajustar los datos.



<font color=violet>Guardamos este df para usarlo en próximas lecciones.

In [27]:
df_resultados.to_pickle('datos/cars_price_LR.pkl')

In [28]:
df_resultados.to_csv('datos/cars_price_LR.csv')